In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
%matplotlib inline

from tqdm import tqdm
import time

import datetime as dt
from datetime import timedelta
from datetime import datetime

from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error

from sklearn.pipeline import make_pipeline, Pipeline

from numpy import arange
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge


from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc, plot_roc_curve, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

#import precision and recall
from sklearn.metrics import precision_score, recall_score

# Trick to widen the screen
from IPython.core.display import display, HTML

#Widens the code landscape 
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows', 50)
sns.set_style('whitegrid')

In [2]:
X_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\All Fields\X_Test.csv")
X_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\All Fields\X_Train.csv")
y_test = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\All Fields\y_Test.csv")
y_train = pd.read_csv(r"C:\Users\Danny Farone\Documents\GitHub\Data-Science-Job-Salaries\data\processed\All Fields\y_Train.csv")

In [3]:
# Create a base classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)


clf = MultiOutputClassifier(clf)
clf.fit(X_train,y_train)

#accuracy of the model on the testing data
clf_score_test = clf.score(X_test, y_test)
#predictions for the model on the testing data
clf_pred_test = clf.predict(X_test)
#Precision of the model on the testing data
precision_test = precision_score(y_test, clf_pred_test, average = 'micro')
#Recall of the model on the testing data
recall_test = recall_score(y_test, clf_pred_test, average = 'micro')

#Print a simple confusion matrix of the teting data results
#print("Initial Decision Tree confusion matrix:")
#print(confusion_matrix(y_test, clf_pred_test), '\n')
#Print the scores we calculated earlier in this block
print("Accuracy: ", clf_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

Accuracy:  0.319672131147541
Sensitivity:  0.319672131147541
Specificity:  0.42857142857142855


In [ ]:
# plot ROC curve for the first draft model
y_pred_prob = clf.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

plt.plot([0, 1], [0, 1],'k--')
plt.plot(fpr, tpr, label='Gradient Boosted Decision Tree')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Decision Tree ROC Curve')
plt.show();

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, clf.predict(X_test))

# roc auc score
log_roc_auc = roc_auc_score(y_test, y_pred_prob)
roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(log_roc_auc)
print(roc_auc_format)

In [ ]:
# Define the hyperparameter space to search
param_grid = {'estimator__n_estimators': [50, 100, 200, 300, 400, 500],
              'estimator__max_features': ['sqrt', 'log2', None],
              'estimator__max_depth': [5, 10, 15, 20, None],
              'estimator__min_samples_split': [2, 5, 10],
              'estimator__min_samples_leaf': [1, 2, 4]}

# Create a grid search object
grid_search = GridSearchCV(estimator=clf,
                           param_grid=param_grid,
                           cv=10,
                           n_jobs=-1)

# Start the timer
start_time = time.time()

# Fit the randomized search object to the training data
with tqdm(total=10) as pbar:
    for i in range(10):
        grid_search.fit(X_train, y_train)
        pbar.update(1)

# Stop the timer
elapsed_time = time.time() - start_time

# Print the best hyperparameters and the corresponding best score
print("Best hyperparameters: ", rand_search.best_params_)
print("Best score: ", rand_search.best_score_)
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

In [ ]:
# fit new model with best parameters determined by gscv on the training data
clf_gs = RandomForestClassifier(n_estimators=71,
                                 max_features='sqrt',
                                 max_depth=20,
                                 min_samples_split=5,
                                 min_samples_leaf=1,
                                 random_state=42)
clf_gs = MultiOutputClassifier(clf_gs)
clf_gs.fit(X_train, y_train)

#accuracy of the model on the testing data
clf_score_test = clf_gs.score(X_test, y_test)
#predictions for the model on the testing data
clf_pred_test = clf_gs.predict(X_test)
#Precision of the model on the testing data
precision_test = precision_score(y_test, clf_pred_test, average = 'micro')
#Recall of the model on the testing data
recall_test = recall_score(y_test, clf_pred_test, average = 'micro')

#Print the scores we calculated earlier in this block
print("Accuracy: ", clf_score_test)
print("Sensitivity: ", recall_test)
print("Specificity: ", precision_test)

In [ ]:
feat_imp = pd.DataFrame(clf_gs.estimators_[0].feature_importances_)
for i in range(1, 4):
    feat_imp[i] = pd.DataFrame(clf_gs.estimators_[i].feature_importances_)

feat_imp = feat_imp.mean(axis=1)
feat_names = pd.DataFrame(list(X.columns))
df_feat_imp = pd.concat([feat_imp, feat_names], axis=1)
df_feat_imp.columns = ['Importance', 'Features']
df_feat_imp.sort_values('Importance', ascending=False, inplace=True)

Importance = pd.DataFrame({'Importance': feat_imp * 100}, index=X.columns)
Importance.sort_values('Importance', axis=0, ascending=True).plot(kind='barh', color='b')
plt.xlabel('Variable Importance')
plt.gca().legend_ = None


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# fit new model with best parameters determined by gscv on the training data
clf_gs = RandomForestClassifier(n_estimators=71,
                                 max_features='sqrt',
                                 max_depth=20,
                                 min_samples_split=5,
                                 min_samples_leaf=1,
                                 random_state=42)
clf_gs = MultiOutputClassifier(clf_gs)
clf_gs.fit(X_train, y_train)

# calculate ROC curve and ROC AUC score for each column of y
for i in range(y_test.shape[1]):
    # calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test[:, i], clf_gs.predict_proba(X_test)[:, i, 1])

    # plot ROC curve
    plt.plot([0, 1], [0, 1],'k--')
    plt.plot(fpr, tpr, label='ROC curve for label {}'.format(i))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Label {}'.format(i))
    plt.show()

    # calculate ROC AUC score
    log_roc_auc = roc_auc_score(y_test[:, i], clf_gs.predict_proba(X_test)[:, i, 1])
    roc_auc_format = 'ROC AUC Score for label {}: {:.4f}'.format(i, log_roc_auc)
    print(roc_auc_format)


In [ ]:
# Get predicted labels on the test set
y_pred = clf_gs.predict(X_test)

# Create a confusion matrix for each target class
for i in range(y_test.shape[1]):
    cm = confusion_matrix(y_test[:,i], y_pred[:,i])
    print(f"Confusion matrix for target {i}:")
    print(cm)


In [ ]:
#use seaborn to create a heatmap of the confusion matrix from above
df_cm = pd.DataFrame(cm)

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(5, 4))
fig.suptitle('Confusion Matrix for Classification of Multiple Claims using Logistic Regression', 
             fontsize=16, y=1.05)
ax = plt.subplot()
sns.heatmap(df_cm, annot=True, cmap="Greens", annot_kws={"size": 16}, ax=ax, fmt="g")
ax.set_xlabel('Predicted', fontsize=15)
ax.set_ylabel('True', fontsize=15)
ax.xaxis.set_ticklabels(['One_Claim', 'Mutliple_Claims'], fontsize=12) 
ax.yaxis.set_ticklabels(['One_Claim', 'Mutliple_Claims'], fontsize=12, va='center')
plt.show()